In [28]:
pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [30]:
#trainフォルダ内のcsvファイルを全て取得
files = glob.glob("train/*.csv")

data_list = []
#filesに格納された４７個のファイルをリストに入れる
for file in files:
    data_list.append(pd.read_csv(file,index_col=0))
#追加したリストデータの結合
df = pd.concat(data_list)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [31]:
def data_pre(df):
    nonnull_list = []

    #カラムを呼び出しnull数を計算、０のものをnonnull_listに追加する
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
            
    df = df.drop(nonnull_list,axis=1) 

    df = df.drop(["市区町村名","種類"],axis=1) 

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)
    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上",2000).astype(float)
    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 34 - num
        if "令和" in i: 
            num = float(i.split("令和")[1].split("年")[0])
            year = 4 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 97 - num
        y_list[i] = year
    y_list["戦前"] = 77
    df["建築年"] = df["建築年"].replace(y_list)
    df["取引時点"]
    year = {
        "年第1四半期":"0.25",
        "年第2四半期":"0.5",
        "年第3四半期":"0.75",
        "年第4四半期":"0.99"
    }

    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep

    df["取引時点"]=df["取引時点"].replace(year_list).astype(float)
    
    for i in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
        df[i] = df[i].astype("category")
    
    return df

In [32]:
df = data_pre(df)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707485 entries, 43046834 to 45025344
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       707485 non-null  int64   
 1   都道府県名         707485 non-null  category
 2   地区名           706818 non-null  category
 3   最寄駅：名称        704776 non-null  category
 4   最寄駅：距離（分）     684350 non-null  float64 
 5   間取り           682328 non-null  category
 6   面積（㎡）         707485 non-null  float64 
 7   建築年           686959 non-null  float64 
 8   建物の構造         689270 non-null  category
 9   用途            640702 non-null  category
 10  今後の利用目的       342163 non-null  category
 11  都市計画          687864 non-null  category
 12  建ぺい率（％）       683596 non-null  float64 
 13  容積率（％）        683596 non-null  float64 
 14  取引時点          707485 non-null  float64 
 15  改装            641456 non-null  category
 16  取引の事情等        18824 non-null   category
 17  取引価格（総額）_log  707485

In [34]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

df_train,df_val = train_test_split(df,test_size=0.2)

col = "取引価格（総額）_log"

train_y = df_train[col]
train_x = df_train.drop(col,axis=1)

val_y = df_val[col]
val_x = df_val.drop(col,axis=1)

trains = lgb.Dataset(train_x,train_y)
valids = lgb.Dataset(val_x,val_y)

params = {
    "objective":"regression",
    "metrics":"mae"
}

model = lgb.train(params,trains,valid_sets=valids,num_boost_round=1000,early_stopping_rounds=100)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.212692 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11675
[LightGBM] [Info] Number of data points in the train set: 565988, number of used features: 17


/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Start training from score 7.226009
[1]	valid_0's l1: 0.25296
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.236542
[3]	valid_0's l1: 0.222223
[4]	valid_0's l1: 0.209567
[5]	valid_0's l1: 0.198204
[6]	valid_0's l1: 0.188414
[7]	valid_0's l1: 0.179439
[8]	valid_0's l1: 0.171828
[9]	valid_0's l1: 0.164945
[10]	valid_0's l1: 0.15881
[11]	valid_0's l1: 0.153223
[12]	valid_0's l1: 0.148252
[13]	valid_0's l1: 0.143878
[14]	valid_0's l1: 0.140168
[15]	valid_0's l1: 0.136602
[16]	valid_0's l1: 0.133321
[17]	valid_0's l1: 0.130475
[18]	valid_0's l1: 0.127794
[19]	valid_0's l1: 0.125448
[20]	valid_0's l1: 0.1231
[21]	valid_0's l1: 0.121065
[22]	valid_0's l1: 0.119003
[23]	valid_0's l1: 0.117344
[24]	valid_0's l1: 0.115642
[25]	valid_0's l1: 0.114114
[26]	valid_0's l1: 0.112667
[27]	valid_0's l1: 0.111249
[28]	valid_0's l1: 0.109938
[29]	valid_0's l1: 0.108863
[30]	valid_0's l1: 0.107279
[31]	valid_0's l1: 0.106399
[32]	valid_0's l1: 0.105458
[3

In [36]:
vals = model.predict(val_x)
mae(vals,val_y)

0.07649557994824342

In [42]:
df_test = pd.read_csv("test.csv",index_col=0)
df_test = data_pre(df_test)

In [44]:
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("submit_test.csv")